In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import os

In [ ]:
model = tf.keras.applications.MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights='imagenet')

In [ ]:
def preprocess_image(image_path, target_size=(128, 128)):
    """Preprocess the image to the required input format for the model."""
    image = Image.open(image_path).convert('RGB')
    image = image.resize(target_size)
    image_array = tf.keras.preprocessing.image.img_to_array(image)
    image_array = np.expand_dims(image_array, axis=0)  # Add batch dimension
    image_array = tf.keras.applications.mobilenet_v2.preprocess_input(image_array)
    return image_array

In [ ]:
def generate_mask(image_array):
    """Generate a rough segmentation mask from the model's predictions."""
    # Run inference
    predictions = model.predict(image_array)
    # Resize back to the original input size
    predictions = tf.image.resize(predictions, (128, 128))
    # Take the argmax across the channels to get the most likely class for each pixel
    mask = tf.argmax(predictions, axis=-1).numpy()[0]
    return mask

In [ ]:
def process_and_save_masks(image_folder, mask_base_folder):
    """Generate and save masks for all images in the specified folder, keeping masks in a separate structure."""
    # Get the class name from the image folder path
    class_name = os.path.basename(image_folder)
    # Define the corresponding mask folder for this class
    mask_folder = os.path.join(mask_base_folder, class_name)
    os.makedirs(mask_folder, exist_ok=True)  # Ensure the mask folder exists
    
    for image_name in os.listdir(image_folder):
        if image_name.endswith(('.jpg', '.jpeg', '.png')):  # Process only image files
            image_path = os.path.join(image_folder, image_name)
            mask_path = os.path.join(mask_folder, image_name)
            
            # Preprocess image and generate mask
            image_array = preprocess_image(image_path)
            mask = generate_mask(image_array)
            
            # Convert the mask to an image and save it
            mask_image = Image.fromarray(np.uint8(mask * 255 / np.max(mask)))  # Scale mask values to 0-255
            mask_image = mask_image.convert("L")  # Convert to grayscale
            mask_image.save(mask_path)

In [ ]:
# Define paths for the original train and validation image directories
train_dir = 'C:/Users/AJ/OneDrive/Desktop/school/Nueral/Neural Jupyter/Project 2/augmented/train'
val_dir = 'C:/Users/AJ/OneDrive/Desktop/school/Nueral/Neural Jupyter/Project 2/augmented/validation'
test_dir = 'C:/Users/AJ/OneDrive/Desktop/school/Nueral/Neural Jupyter/Project 2/augmented/test'

# Define paths for the new mask directories
train_mask_dir = 'C:/Users/AJ/OneDrive/Desktop/school/Nueral/Neural Jupyter/Project 2/masks_augmented/train'
val_mask_dir = 'C:/Users/AJ/OneDrive/Desktop/school/Nueral/Neural Jupyter/Project 2/masks_augmented/validation'
test_mask_dir = 'C:/Users/AJ/OneDrive/Desktop/school/Nueral/Neural Jupyter/Project 2/masks_augmented/test'

In [ ]:
# Iterate over each class folder and generate masks for train and validation sets
for split_dir, mask_split_dir in [(test_dir, test_mask_dir)]:
    for class_name in os.listdir(split_dir):
        class_image_dir = os.path.join(split_dir, class_name)
        if os.path.isdir(class_image_dir):
            # Define the corresponding mask folder
            class_mask_dir = os.path.join(mask_split_dir, class_name)
            os.makedirs(class_mask_dir, exist_ok=True)
            
            # Generate and save masks for each image in this class folder
            process_and_save_masks(class_image_dir, class_mask_dir)